# Jedgment Summarization



## 安裝套件 (For google collab)


In [3]:
# ! pip install transformers
# ! pip install datasets
# ! pip install torcheval
# ! pip install scikit-learn
# ! pip install torch
# ! pip install pytorch-ignite
# ! pip install ipywidgets
# ! pip install jieba

  Using cached scipy-1.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 37.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 18.2 MB/s eta 0:00:00
Using cached scipy-1.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)


In [21]:
import transformers as T
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from ignite.metrics import Rouge
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import jieba
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## 超參數
 - 學習率 (learning rate): 1e-4
 - 訓練輪數 (epochs): 10
 - 優化器 (optimizer): AdamW
 - 批次大小 (batch size): 8
 - 評量指標 (evaluation matrics)Rouge-2

In [22]:
lr = 1e-4
epochs = 10
train_batch_size = 1
validation_batch_size = 1
test_batch_size = 1
random_seed = 42
max_length = 300

# path setting that souldn't be changed
tokenizer_path = "./saved_tokenizer"
dataset_path = "./train.xlsx"
test_dataset_path = './test.xlsx'
model_path = "./saved_models"

## 載入BART模型
 - 使用huggingface裝載模型的架構、參數和tokenizer
 - 保存在路徑./cache/中
 - 用.to(device)把模型裝載入訓練設備(GPU)
 - 使用的 tokenizer 請儲存到指定路徑

In [23]:
model = T.AutoModelForSeq2SeqLM.from_pretrained("IDEA-CCNL/Randeng-BART-139M-SUMMARY", cache_dir="./cache/").to(device)
tokenizer = T.AutoTokenizer.from_pretrained("IDEA-CCNL/Randeng-BART-139M-SUMMARY", cache_dir="./cache/", model_max_length=512)
tokenizer.save_pretrained(tokenizer_path)

c:\Users\randy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./saved_tokenizer\\tokenizer_config.json',
 './saved_tokenizer\\special_tokens_map.json',
 './saved_tokenizer\\tokenizer.json')

In [24]:
optimizer = AdamW(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

## 資料處理
 - 使用 torch.utils.data 中的 Dataset 和 Dataloader 成批次地讀取和預處理資料

In [25]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = tokenizer.batch_encode_plus([each["origin_context"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    model_outputs = tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device)

class Legal_Judgment_Dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        data_df = pd.read_excel(dataset_path)
        self.data = []
        for index, row in data_df.iterrows():
            origin_context = "summary: " + str(row['裁判原文'])
            summary = row['摘要']
            self.data.append({"origin_context": origin_context, "summary": summary})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


- 在範例程式中有切分成 train, valid, test，實作上可將全部資料用於 train 與 valid

In [ ]:
dataset = Legal_Judgment_Dataset()
train_dataset, untrain_dataset = train_test_split(dataset, test_size=0.2, random_state=random_seed)
test_dataset, validation_dataset = train_test_split(untrain_dataset, test_size=0.5, random_state=random_seed)
summary_train = DataLoader(train_dataset, collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
summary_validation = DataLoader(validation_dataset, collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)

rouge = Rouge(variants=["L", 2], multiref="best")
print(f"Dataset example: \n{train_dataset[0]} \n{train_dataset[1]} \n{train_dataset[2]}")

## 驗證
驗證程式
 - 將驗證資料輸入模型，用Rouge-2評價輸出的效果
 - Rouge的使用方法參考 https://pytorch.org/ignite/generated/ignite.metrics.Rouge.html

In [27]:
def evaluate(model, dataset):
    pbar = tqdm(dataset)
    pbar.set_description(f"Evaluating")
    loss_list = []
    for inputs, targets in pbar:
        loss = model(inputs, labels=targets).loss
        loss_list.append(loss.item())
        pbar.set_postfix(loss = loss.item())
        outputs = [each.replace("<unk>", "").replace("<pad>","") for each in tokenizer.batch_decode(model.generate(inputs), max_length=max_length)]
        targets = [each.replace("<unk>", "").replace("<pad>","") for each in tokenizer.batch_decode(targets)]
        print("model generate examples:")
        print(f"output: {outputs}")
        print(f"target: {targets}")
        for out, tar in zip(outputs, targets):
            sentence = " ".join(jieba.cut(out)).split()
            ground_truth = " ".join(jieba.cut(tar)).split()
            for s in sentence:
                rouge.update(([s], [ground_truth]))
            
    return rouge.compute(), np.mean(np.array(loss_list))



## 訓練
 - 將資料成批次輸入BART模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [ ]:
for ep in range(epochs):
    model.train()
    pbar = tqdm(summary_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = model(input_ids=inputs, labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    torch.save(model, f'{model_path}/ep{ep}.mod')
    model.eval()
    validation_score, validation_loss = evaluate(model, summary_validation)
    scheduler.step(validation_loss)
    print(f"Rouge-2 score on epoch {ep}:", validation_score)

## 測試
- 使用以下程式接跑 test data

In [29]:
def test(model, test_dataset):
    model.eval()
    pbar = tqdm(test_dataset)
    pbar.set_description(f"Testing")

    for idx, testdata in tqdm(test_dataset.iterrows(), total=test_dataset.shape[0]):
        ori_data = "summary: " + str(testdata["裁判原文"])
        inputs = tokenizer(ori_data, padding=True, truncation=True, return_tensors="pt").to(device)
        outputs_context = tokenizer.decode(model.generate(**inputs)[0], max_length=max_length)
        outputs = outputs_context.replace("<unk>", "").replace("<pad>","")
        test_dataset.loc[idx, "摘要"] = outputs
            
    return test_dataset


In [ ]:
test_dataset = pd.read_excel(test_dataset_path)
test_dataset['摘要'] = None
result_data = test(model, test_dataset)
result_data.to_excel("result.xlsx", index=False)